# Delayed tasks analysis


In [1]:
# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *
from trialexp.utils.data_organisation import *


c:\Users\phar0732\.conda\envs\trialexp\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\phar0732\.conda\envs\trialexp\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\phar0732\.conda\envs\trialexp\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Variables

In [2]:
import pandas as pd
from pathlib import Path
trial_window = [-2000, 6000] # in ms

# time limit around trigger to perform an event
# determine successful trials
# timelim = [1000, 4000] # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder = Path(os.getcwd()).parent.parent

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = Path(basefolder,'params','tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv' 

# from sample_data

# # From jade
# photometry_dir = '/home/MRC.OX.AC.UK/phar0732/ettin/Data/head-fixed/photometry'
# pycontrol_dir = '/home/MRC.OX.AC.UK/phar0732/ettin/Data/head-fixed/pycontrol'

# # # From julien-pc
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\_Other\test_folder\delayed_go\pyphotometry\delayed_go_dual_2022'
pycontrol_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\_Other\test_folder\delayed_go\pycontrol\delayed_go_dual_2022'

#From laptop
# photometry_dir = r'C:\Users\phar0732\Documents\GitHub\trialexp\sample_data\pyphotometry\reaching_go_spout_incr_break2_nov22'
# pycontrol_dir = r'C:\Users\phar0732\Documents\GitHub\trialexp\sample_data\pycontrol\reaching_go_spout_incr_break2_nov22'

video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'
tasks = pd.read_csv(tasksfile, usecols=[1, 2, 3, 4], index_col=False)


### Create an experiment object


In [3]:
# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(path=pycontrol_dir, int_subject_IDs=True, update=True, verbose=False)

### Find matching files 

- pycontrol
- pyphotometry
- videos
- deeplabcut outputs

In [4]:
exp_cohort = match_sessions_to_files(exp_cohort, pycontrol_dir, ext='txt')
exp_cohort = match_sessions_to_files(exp_cohort, photometry_dir, ext='ppd')
exp_cohort = match_sessions_to_files(exp_cohort, video_dir, ext='mp4')
exp_cohort = match_sessions_to_files(exp_cohort, video_dir, ext='h5')

### Reorganisation of data by single session folders, including all files from all data modalities

- sessions_folder is 

In [5]:

sessions_folder = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\_Other\test_folder\by_session_folder'

 # if multiple streams for the data modality (e.g. 2 cameras / 2 probes etc.)
multi_stream_keywords = {
    'video': ('Side','Down'),
    'dlc': ('Side','Down'),
    'lfp': ('probeA','probeB'), # Not yet implemented
    'spikes': ('probeA','probeB') # Not yet implemented
    }                               # Should be usable for multi-site photometry


reorg_to_sessions_folder(exp_cohort, sessions_folder, multi_stream_keywords)

In [24]:
from pathlib import Path
import os
from itertools import compress

path = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\_Other\test_folder\by_session_folder'

path = Path(path)
session_folder_list = os.listdir(path)
# check if all are folders
is_folder = [os.path.isdir(path / s_path) for s_path in session_folder_list]
# only integrate folders
session_folder_list = list(compress(session_folder_list, is_folder))

pycontrol_folders = [path / s_f / 'pycontrol' for s_f in session_folder_list]
pycontrol_files = [f / os.listdir(f)[0] for f in pycontrol_folders if
    (len(os.listdir(f)) == 1) and (os.listdir(f)[0][-4:] == '.txt')]

In [45]:
exp_session_folders = Experiment(path, update=True, folder_by_session=True)

In [46]:
exp_session_folders.process_exp_by_trial(tasksfile = tasksfile, trial_window=[-2000, 8000])

In [ ]:
exp_session_folders.